In [1]:
import re
import copy 
import math
import itertools
import jellyfish
from tqdm import tqdm
import pandas as pd
import numpy as np
from datetime import datetime
import xml.etree.ElementTree as ET

ns = {'xml': 'http://www.w3.org/XML/1998/namespace',
      'dflt': 'http://www.tei-c.org/ns/1.0',
      'frus':'http://history.state.gov/frus/ns/1.0',
      'xi':'http://www.w3.org/2001/XInclude'
      }

In [2]:
def extract_city(doc):

    # city
    place_tag = doc.find('.//dflt:placeName',ns)
    if place_tag is not None:
        city = place_tag.text # use iter text instead of this!!!
        #city = "".join(place_tag.itertext())
    else:
        city = None

    global city_df
    city_df = pd.concat((city_df, pd.DataFrame({'name':[city]})),ignore_index=True)
    return

In [3]:
import glob
volume_root = 'frus1969-76'

city_df = pd.DataFrame(columns=['name'])


#for file in glob.glob('volumes/*'):
for file in glob.glob('volumes/'+volume_root+'*'):

    tree = ET.parse(file)
    root = tree.getroot()

    docs = root.findall('./dflt:text/dflt:body//dflt:div[@type="document"]', ns)
    for doc in docs:
        extract_city(doc)


In [ ]:
#pd.set_option('display.max_rows', None)

In [4]:
city_df.dropna(inplace=True)
city_df.drop_duplicates(inplace=True)
city_df.reset_index(drop=True,inplace=True)

In [6]:
city_df

,name
0,Washington
1,Moscow
2,Vienna
3,Camp David
4,Salzburg White House
...,...
241,Sanaa
242,Bern
243,Washingon
244,"Rambouillet, France"


In [7]:
all_names = city_df['name'].values

def compute_sim(s1,func,s2):
    return func(s1,s2)

def find_matches(s2):

    spiro_dist_df = pd.DataFrame({'name_set':all_names,
                                'dam_lev_dist':[compute_sim(x, jellyfish.damerau_levenshtein_distance,s2) for x in all_names]})
    
    # addition to original matching criteria
    misspelling_idx = set(spiro_dist_df[(spiro_dist_df['dam_lev_dist'] <=1)].index.values)

    return misspelling_idx

In [8]:
t = {}
for idx in tqdm(range(len(all_names))):
    name = all_names[idx]
    t[idx]=find_matches(name)

100%|██████████| 246/246 [00:00<00:00, 839.69it/s]


In [10]:
scratch_t = copy.deepcopy(t)
changed_flag = True

while changed_flag:

    changed_flag = False

    for key in t:
        
        for matched_idx in t[key]:

            if key != matched_idx:
                if scratch_t.get(key, None) and scratch_t.get(matched_idx, None):
                    changed_flag = True
                    t[key] = t[key].union(t[matched_idx])
                    scratch_t.pop(matched_idx, None)
        
    unwanted = set(t.keys()) - set(scratch_t.keys())
    print(f'removing {len(unwanted)} keys.')
    for unwanted_key in unwanted: del t[unwanted_key]
    scratch_t = copy.deepcopy(t)
    print('---')
    

removing 13 keys.
---
removing 1 keys.
---
removing 0 keys.
---


In [11]:
for temp_key in t:
    
    te_df = city_df.iloc[list(t[temp_key])]

    name_list = te_df['name'].values

    city_df.at[temp_key, 'name_list'] = name_list

city_df = city_df.loc[t.keys()]

In [13]:
city_df

,name,name_list
0,Washington,"[Washington, Washingon, Washington,, Washingto..."
1,Moscow,[Moscow]
2,Vienna,[Vienna]
3,Camp David,[Camp David]
4,Salzburg White House,[Salzburg White House]
...,...,...
239,"Trois Ilets, Martinique","[Trois Ilets, Martinique]"
240,Dhahran,[Dhahran]
242,Bern,[Bern]
244,"Rambouillet, France","[Rambouillet, France]"


### wikidata matching

In [16]:
import pandas as pd
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
from SPARQLWrapper import SPARQLWrapper, JSON

from tqdm import tqdm
tqdm.pandas()

import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [17]:
user_agent = 'CoolBot/0.0 (https://example.org/coolbot/; coolbot@example.org)'

sparqlwd = SPARQLWrapper("https://query.wikidata.org/sparql", agent=user_agent)
sparqlwd.setReturnFormat(JSON)

In [20]:
def find_wiki_entity(name):

    try:
        query = """
        SELECT ?country ?countryLabel WHERE {
        SERVICE wikibase:mwapi {
            bd:serviceParam wikibase:endpoint "www.wikidata.org";
                            wikibase:api "EntitySearch";
                            mwapi:search  \'"""+name+"""\';
                            mwapi:language "en".
            ?city wikibase:apiOutputItem mwapi:item.
            ?num wikibase:apiOrdinal true.
        }
        ?city (wdt:P31/wdt:P279*) wd:Q486972.
        #?city wdt:P31 wd:Q5119.
        ?city wdt:P17 ?country.
        SERVICE wikibase:label { bd:serviceParam wikibase:language "en".}
        }
        """
        
        sparqlwd.setQuery(query)

        return sparqlwd.query().convert()

    except Exception as e:
        print(f'name: {name}')
        print(f'error message: {e}')
        return {'head': {'vars': ['item']}, 'results': {'bindings': []}}


def process_name_list(row):

    name_list = row['name_list']

    wiki_tag = set()

    for name in name_list:
        res = find_wiki_entity(name)

        for binding in res['results']['bindings']:
            wiki_tag.add(binding['item']['value'])

    return list(wiki_tag)

In [26]:
res = find_wiki_entity('Barcelona')

In [47]:
from collections import Counter
candidates = []

for binding in res['results']['bindings']:
    candidates.append(binding['countryLabel']['value'])

selected_country = Counter(candidates).most_common(1)[0][0]
wiki_tag = None

for binding in res['results']['bindings']:
    if binding['countryLabel']['value'] == selected_country:
        wiki_tag = binding['country']['value']
        break

In [49]:
wiki_tag

'http://www.wikidata.org/entity/Q29'

In [ ]:
wiki_col = new_unified_person_df.progress_apply(lambda x: process_name_list(x),axis=1)

In [ ]:
new_unified_person_df['wiki_col'] = wiki_col